# Higher-order Tensor Renormalization Group for 2D Ising model

In [28]:
import numpy as np
from numpy.typing import ArrayLike
import hotrg
import ipywidgets as widgets
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ["svg"]
import itertools
from tqdm.auto import tqdm
import cProfile
from numba import jit

The average **magnetization** $\braket{m}$ of a statistical system can be computed as,

$$ \langle\text{Magnetization}\rangle = \frac{\text{Tr}(H)}{Z} $$

The **partition function** $Z$ can be described as a translation-invariant tensor network contraction.

$$ Z = \sum_{i} e^{- \beta E_i} = \text{Tr} \prod_i T_{l_i r_i u_i d_i}(\beta) $$

where

$$
W = \begin{pmatrix}
\sqrt{\cosh(1/t)} & \sqrt{\sinh(1/t)} \\
\sqrt{\cosh(1/t)} & -\sqrt{\sinh(1/t)}
\end{pmatrix}
$$

$$
T_{lrud} = \sum_\alpha W_{\alpha,l} W_{\alpha,r} W_{\alpha,u} W_{\alpha,d}
$$

The $H$ tensor has a similar definition to $T$.
$$ H^{(0)}_{lrud} = \sum_\alpha W_{\alpha,l} ~ W_{\alpha,r} ~ W_{\alpha,u} ~ W_{\alpha,d} ~ m_\alpha ~~~~ \text{where} ~~~~ m_\alpha = \begin{pmatrix} 1 \\ -1 \end{pmatrix} $$

## Procedure

1. Contract $T$ with the tensor below and reshape to a 4-order tensor $M$.

$$ M^{(n)}_{lrud} = \sum_\alpha T^{(n-1)}_{l_1 r_1 u \alpha} T^{(n-1)}_{l_2 r_2 \alpha d} ~~~~ \text{where} ~~~~ l = l_1 \otimes l_2, ~~ r = r_1 \otimes r_2 $$

In [3]:
def contract_tensors(top: np.ndarray, bottom: np.ndarray) -> np.ndarray:
	m = np.einsum("abci,deif->adbecf", top, bottom)
	return m.reshape((m.shape[0]*m.shape[1], m.shape[2]*m.shape[3], m.shape[4], m.shape[5]))


2. Perform the HOSVD on $M$.

$$ M^{(n)}_{lrud} = \sum_{\alpha \beta \gamma \epsilon} S_{\alpha \beta \gamma \epsilon} U^L_{l,\alpha} U^R_{r,\beta} U^U_{u,\gamma} U^D_{d,\epsilon} $$

In [4]:
def hosvd_sides(a: ArrayLike) -> tuple[ArrayLike, list[ArrayLike]]:
    """Higher-order singular value decomposition (HOSVD)

    parameters
    ----------
    - a. ArrayLike

    returns
    -------
    - S. ArrayLike
    - U. Sequence[ArrayLike]
    """

    Us = []
    Ak = a

    for k in range(a.ndim):
        # fold Ak
        Ak = np.moveaxis(Ak, k, 0)
        shape = list(Ak.shape)
        Ak = np.reshape(Ak, (Ak.shape[0], -1))

        # compute SVD
        u, s, vh = np.linalg.svd(Ak, full_matrices=False, compute_uv=True, hermitian=False)
        shape[0] = len(s)

        # unfold Ak for next iteration
        Ak = np.atleast_2d(s).T * vh
        Ak = np.reshape(Ak, shape)

        Us.append(u)

    Ak = Ak.transpose()
    return (Ak, Us)


3. Compute the truncation error from sides _L_ and _R_.

$$
\varepsilon_L = \sum_{i > \chi} \vert S_{i,:,:,:} \vert^2
$$

$$
\varepsilon_R = \sum_{j > \chi} \vert S_{:,j,:,:} \vert^2
$$


In [14]:
def error_left(S: np.ndarray, D: int) -> float:
	return sum(np.sum(np.square(np.abs(S[i,:]))) for i in range(D, S.shape[0]))

def error_right(S: np.ndarray, D: int) -> float:
	return sum(np.sum(np.square(np.abs(S[:,j,:]))) for j in range(D, S.shape[1]))


4. Create a projector $U$ from the side that better approximates $M$, and normalize.
   - Normalizing against $\sqrt{\max(S)}$ seems to work.
   - $\sqrt{}$ is due to the fact that the projector is applied twice.
   - If max bond dimension $\chi$ is surpassed, truncate.

$$
\hat{U}^{(n)} = \begin{cases}
U^L / \sqrt{\max{S}} & \varepsilon_1 < \varepsilon_2 \\
U^R / \sqrt{\max{S}} & \text{otherwise}
\end{cases}
$$


5. Project $M$ with $\hat{U}$ to update $T$.

$$ T^{(n)}_{lrud} = \sum_{\alpha \beta} \hat{U}_{l,\alpha} M^{(n)}_{\alpha \beta u d} \hat{U}_{r,\beta} $$

In [29]:
def update_tensor(U: np.ndarray, M: np.ndarray) -> np.ndarray:
    return np.einsum("il,ijud,jr->lrud", U, M, U)

In [7]:
def one_iteration(H: np.ndarray, T: np.ndarray, chi: int) -> tuple[np.ndarray, np.ndarray]:
	# contract with environment
	Mh = contract_tensors(H, T)
	Mt = contract_tensors(T, T)

	# decompose tensor
	S, Us = hotrg.hosvd(Mt)

	# compute projector
	epsl = error_left(S, chi)
	epsr = error_right(S, chi)
	U = Us[0 if epsl < epsr else 1]

	if U.shape[1] > chi:
		U = U[:, 0:chi]
	U /= np.sqrt(np.max(S))

	# update tensors
	H = update_tensor(U, Mh)
	T = update_tensor(U, Mt)

	return H, T

## Benchmark version

In [8]:
temperature = widgets.FloatSlider(min=2, max=5)
temperature

FloatSlider(value=2.0, max=5.0, min=2.0)

In [9]:
repeats = widgets.IntSlider(min=10, max=20)
repeats

IntSlider(value=10, max=20, min=10)

In [13]:
chi = widgets.IntSlider(value=8, min=8, max=24)
chi

IntSlider(value=8, max=24, min=8)

In [11]:
def trace(t):
	return np.einsum("iijj->", t)

### benchmark without transposition

In [35]:
T = hotrg.ising.partition_tensor(temperature.value)
H = hotrg.ising.magnetization(temperature.value)
magnetization = [trace(H)/trace(T)]

with cProfile.Profile() as profile:
	for _ in tqdm(range(repeats.value)):
		for permutator in itertools.islice(itertools.cycle([(0,1,3,2), (2,3,0,1)]), 4):
				H, T = one_iteration(H, T, chi.value)
				H = H.transpose(permutator)
				T = T.transpose(permutator)
		print(f"magnetization = {trace(H)/trace(T)}")

  0%|          | 0/10 [00:00<?, ?it/s]

magnetization = -2.159864732248949e-16
magnetization = -2.6555823783992064e-15
magnetization = -1.490200049181677e-13
magnetization = -7.815866879503736e-08
magnetization = -1.3994438237276997e-06
magnetization = -2.2553406451689145e-05
magnetization = -0.0003617147569484757
magnetization = -0.005788923359692794
magnetization = -0.09230833327942953
magnetization = -0.8434603475834389


In [37]:
profile.print_stats("tottime")

         181920 function calls (172553 primitive calls) in 220.629 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      180  155.461    0.864  155.461    0.864 {built-in method numpy.core._multiarray_umath.c_einsum}
      160   61.682    0.386   62.065    0.388 linalg.py:1470(svd)
     4800    0.813    0.000    0.920    0.000 2430219045.py:2(<genexpr>)
      400    0.681    0.002    0.681    0.002 {method 'reshape' of 'numpy.ndarray' objects}
       40    0.636    0.016   63.071    1.577 hosvd.py:7(hosvd)
     4800    0.544    0.000    0.636    0.000 2430219045.py:5(<genexpr>)
      480    0.379    0.001    0.379    0.001 {method 'astype' of 'numpy.ndarray' objects}
     9560    0.155    0.000    0.155    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       40    0.091    0.002  220.549    5.514 3288473989.py:1(one_iteration)
     9560    0.020    0.000    0.184    0.000 fromnumeric.py:69(_wrapreduction)
     9520    0.

In [30]:
profile.clear()